In [1]:
# This cell is NOT editable. Overwrite variables on your own discretion.
# Any changes other than the script code will NOT BE SAVED!
# All cells are assumed to be script code cells, unless explictly tagged as 'o9_ignore'

In [2]:
# Slice Dimension Attributes defined in the plugin. Please check all queries and replace <KEY HERE> with a valid name.
# For example: If slice is defined by Version.[Version Name] and Time.[Month]
# input_df = ibpl Select ([Version].[Version Name].[<KEY HERE>] * [Time].[Month].[<KEY HERE>] * [Item].[Item Number]) on row, ({Measure.[M1], Measure.[M2]}) on column limit 5000;
#                             update <KEY HERE> to valid names
# input_df = ibpl Select ([Version].[Version Name].[CurrentWorkingView] * [Time].[Month].[January] * [Item].[Item Number]) on row, ({Measure.[M1], Measure.[M2]}) on column limit 5000;

_input_df = "select (Version.[Version Name]*Product.[Product]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"
_predict_df = "select (Version.[Version Name]*Product.[Product]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"


# Initialize the O9DataLake with the input parameters and dataframes
# Data can be accessed with O9DataLake.get(<Input Name>)
# Overwritten values will not be reflected in the O9DataLake after initialization

from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource
input_df = O9DataLake.register("input_df",DataSource.LS, ResourceType.IBPL, _input_df)
predict_df = O9DataLake.register("predict_df",DataSource.LS, ResourceType.IBPL, _predict_df)

2023-01-30 05:48:36,165 : srishtisingh_5502 :IbplClient: WebApi Request Url = [https://myazureqatest.o9solutions.com/api/ibplquery/5502/ExecuteCompactJsonQuery]
2023-01-30 05:48:36,169 : srishtisingh_5502 :Starting new HTTPS connection (1): myazureqatest.o9solutions.com:443
2023-01-30 05:48:36,655 : srishtisingh_5502 :https://myazureqatest.o9solutions.com:443 "POST /api/ibplquery/5502/ExecuteCompactJsonQuery HTTP/1.1" 200 None
2023-01-30 05:48:41,665 : srishtisingh_5502 :IbplClient: WebApi Request Url = [https://myazureqatest.o9solutions.com/api/ibplquery/5502/ExecuteCompactJsonQuery]
2023-01-30 05:48:41,668 : srishtisingh_5502 :Starting new HTTPS connection (1): myazureqatest.o9solutions.com:443
2023-01-30 05:48:41,893 : srishtisingh_5502 :https://myazureqatest.o9solutions.com:443 "POST /api/ibplquery/5502/ExecuteCompactJsonQuery HTTP/1.1" 200 None
2023-01-30 05:48:46,921 : srishtisingh_5502 :IbplClient: WebApi Request Url = [https://myazureqatest.o9solutions.com/api/ibplquery/5502/Ex

In [3]:
"""
exec plugin instance [CsvMGIP1_CsvMGIP2_CsvMGPrimDf_CsvMGOP1_Slice] for measures {[DPSellOutUnitsActuals]} using scope ([Product].[Product].Filter(#.Name in {[196426],[208821]}) * [Time].[Day] * [Version].[Version Name].[CurrentWorkingView]);

"""
import logging
from sklearn import tree
import logging
logger = logging.getLogger('o9_logger')
logger.info("######################################")
logger.error(f'Ritu Predict df {predict_df.shape}')
logger.error(f'Ritu input df {input_df.shape}')
logger.error(f'Ritu Predict df count {predict_df.count()}')
logger.error(f'Ritu input df count {input_df.count()}')
logger.error(f'Ritu Predict df columns {predict_df.columns}')
logger.error(f'Ritu input df columns {input_df.columns}')
logger.info("######################################")

out_df = None

if predict_df is not None and len(predict_df.index) > 0:
    x = input_df[['Mean Pricing Save PCT', 'Placement Count', 'Promotion Count', 'DPSellOutPrice']]
    y = input_df[['DPSellOutUnitsActuals']]
    x = x.fillna(0).values
    y = y.fillna(0).values

    clf = tree.DecisionTreeRegressor()
    clf = clf.fit(x, y)

    predict = predict_df[['Mean Pricing Save PCT', 'Placement Count', 'Promotion Count', 'DPSellOutPrice']].fillna(0).values
    # predict_keys = predict_df[['Time.[FiscalWeek]', 'Product.[Product]', 'Version.[Version Name]', 'SalesAccount.[Account]',                               'Location.[Location]']]
    # predict_keys["Product.[Product]"] = pd.to_numeric(predict_keys["Product.[Product]"], downcast='integer')
    predict_values = clf.predict(predict)
    predict_df["DPSellOutUnitsFcst"] = predict_values
    #out_df = pd.concat([predict_keys, out_df_part], axis=1)
    out_df=predict_df[['Time.[FiscalWeek]', 'Product.[Product]', 'Version.[Version Name]', 'SalesAccount.[Account]','Location.[Location]','DPSellOutUnitsFcst']]
    logger.info(out_df)

else:
    out_df = None


2023-01-30 05:48:57,646 : srishtisingh_5502 :######################################
2023-01-30 05:48:57,647 : srishtisingh_5502 :Ritu Predict df (167, 10)
2023-01-30 05:48:57,648 : srishtisingh_5502 :Ritu input df (167, 10)
2023-01-30 05:48:57,653 : srishtisingh_5502 :Ritu Predict df count Version.[Version Name]    167
Product.[Product]         167
Time.[FiscalWeek]         167
SalesAccount.[Account]    167
Location.[Location]       167
DPSellOutUnitsActuals      75
Mean Pricing Save PCT      21
Placement Count           160
Promotion Count            21
DPSellOutPrice             72
dtype: int64
2023-01-30 05:48:57,655 : srishtisingh_5502 :Ritu input df count Version.[Version Name]    167
Product.[Product]         167
Time.[FiscalWeek]         167
SalesAccount.[Account]    167
Location.[Location]       167
DPSellOutUnitsActuals      75
Mean Pricing Save PCT      21
Placement Count           160
Promotion Count            21
DPSellOutPrice             72
dtype: int64
2023-01-30 05:48:5